#Hopfield
Design a Hopfield Network and train it on the images of the first 10 letters of English alphabet to get
a noisy image and correct it. In order to test model, add noise to X percent of each of the images
and then give them to your network. In the end calculate the network’s accuracy.

In [ ]:
!wget https://noto-website-2.storage.googleapis.com/pkgs/NotoSans-hinted.zip
!unzip NotoSans-hinted.zip
!mv NotoSans-Light.ttf  /usr/share/fonts/truetype/
!mkdir fonts
!mv *.ttf ./fonts


--2020-06-14 06:20:11--  https://noto-website-2.storage.googleapis.com/pkgs/NotoSans-hinted.zip
Resolving noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)... 108.177.111.128, 2607:f8b0:4001:c07::80
Connecting to noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16796031 (16M) [application/zip]
Saving to: ‘NotoSans-hinted.zip’

NotoSans-hinted.zip 100%[===================>]  16.02M  --.-KB/s    in 0.1s    

2020-06-14 06:20:11 (117 MB/s) - ‘NotoSans-hinted.zip’ saved [16796031/16796031]

Archive:  NotoSans-hinted.zip
  inflating: LICENSE_OFL.txt         
  inflating: NotoSans-Black.ttf      
  inflating: NotoSans-BlackItalic.ttf  
  inflating: NotoSans-Bold.ttf       
  inflating: NotoSans-BoldItalic.ttf  
  inflating: NotoSans-Condensed.ttf  
  inflating: NotoSans-CondensedBlack.ttf  
  inflating: NotoSans-CondensedBlackItalic.ttf  
  inflat

In [ ]:
import random
import re
import os
import numpy as np
import tkinter as tk  
import matplotlib.font_manager as fm

from matplotlib import pyplot as plt
from PIL import Image, ImageFont
from tkinter import ttk, font  
from scipy import misc

In [ ]:
fontpath = '/usr/share/fonts/truetype/NotoSans-Light.ttf'
fontprop = fm.FontProperties(fname=fontpath, size= 40)

In [ ]:
def create_train_images(font_size):
    imgs = {}
    shape = {}
    font = ImageFont.truetype(fontpath, font_size)

    for char in alphabets:
        img =  Image.Image()._new(font.getmask(char, mode='L'))
        img.save("train - " + str(font_size)+ " - "+ char + ".jpg")
        shape[char] = img.size
        imgs[char] = img 
    imgs_shape[font_size] = shape    
    return imgs

In [ ]:
def create_test_images(font_size):
    imgs = {}

    for noise in noises:
        imgs[noise] = {}

        for char in alphabets:
            img = np.array(train_imgs[font_size][char])
            
            pixels_count = img.shape[0] * img.shape[1] 
            noisypx_count = int(noise * pixels_count)

            random_pxs = random.sample(range(0, pixels_count), noisypx_count)
            
            for i in random_pxs:
                row = int(i / img.shape[1]) 
                column = int(i % img.shape[1]) 

                img[row,column] = 255 if img[row,column] < 125  else 0 
            
            img = Image.fromarray(img, mode="L")
            imgs[noise][char] = img
            img.save("test -" + str(font_size) + " - noise - "+ str(noise)+ " - "+ char + " - "+ ".jpg")
            
           
    return imgs

In [ ]:
def create_data():

   for fsize in font_sizes:
        train_imgs[fsize] = create_train_images(fsize)
        test_imgs[fsize] = {}
        test_imgs[fsize] = create_test_images(fsize)   


In [ ]:
# Read Image file and convert it to Numpy array
def image_to_array(image, threshold = 45):
    image =image.resize((100, 100))
    image = np.array(image)
    y= np.zeros(image.shape, dtype=np.float)
    y[image > threshold] = 1
    y[y == 0] = -1
    return y
    
# convert matrix to a vector
def matrix_to_vector(x):
    return x.flatten()


# Create Weight matrix for a single image
def create_matrix_w(x):
    w = np.zeros([len(x),len(x)])
    for i in range(len(x)):
        for j in range(i,len(x)):
            if i == j:
                w[i, j] = 0
            else:
                w[i, j] = x[i]*x[j]
                w[j, i] = w[i, j]
    return w

# Convert Numpy array to Image file like Jpeg
def array_to_img(data, font_size, char):
    y = np.zeros(data.shape, dtype=np.uint8)
    y[data == 1] = 255
    y[data == -1] = 0
    img = Image.fromarray(y, mode="L")
    # img = img.resize(imgs_shape[font_size][char])
    return img


def update(w, y_vec, theta=0.5, time=100):
    for s in range(time):
        i = random.randint(0, len(y_vec) - 1)
        u = np.dot(w[i][:], y_vec) - theta
        y_vec[i] = 1 if u > 0 else -1

    return y_vec


In [ ]:
# Initial setting
def hopfield(font,train_files, test_files,
             theta=0.5, time=1000):
    
    # read image and convert it to Numpy array
    print("Importing images and creating weight matrix....")

    # num_files is the number of files -- Training
    w = []
    for char in alphabets:
        print (char, "   ", "font : ", font)
        x = image_to_array(train_files[char])
        x_vec = matrix_to_vector(x)
        
        if char == 'A':
            w = create_matrix_w(x_vec)
        else:
            w = w + create_matrix_w(x_vec)

    print(w)

    print("Weight matrix is done!!")

    #Import test data -- testing
    for noise in noises :
        for char in alphabets:
            y = image_to_array(test_files[noise][char])

            oshape = y.shape
            print( "test data : ", "noise = ", noise,"font :",   font )

            y_vec = matrix_to_vector(y)
            
            y_vec_after = update(w=w, y_vec=y_vec, theta=theta, time=time)
            y_vec_after = y_vec_after.reshape(oshape)
            
            newimage = array_to_img(y_vec_after, font, char)
            newimage.save("output -"+str(font ) + "- noise - "+ str(noise)+ " - "+ char + " - "+ ".jpg")


In [ ]:
def main():
    
    create_data()
    print("data creation finished  \n")   
    

    for font in font_sizes:
        hopfield(font, train_files=train_imgs[font], test_files=test_imgs[font], 
                theta=0.5, time=20000)

In [ ]:
Round = 0

In [ ]:
alphabets = "ABCDEFGHIJ"
font_sizes = [16, 32, 64]
noises = [0.1, 0.3, 0.6]

train_imgs = {}  
test_imgs = {}
imgs_shape = {}

main()
# ! unzip images2.zip
! zip images{tryRound}.zip *.jpg
! rm *.jpg
Round += 1

data creation finished  

Importing images and creating weight matrix....
A     font :  16
B     font :  16
C     font :  16
D     font :  16
E     font :  16
F     font :  16
G     font :  16
H     font :  16
I     font :  16
J     font :  16
[[ 0. 10. 10. ...  8.  8.  8.]
 [10.  0. 10. ...  8.  8.  8.]
 [10. 10.  0. ...  8.  8.  8.]
 ...
 [ 8.  8.  8. ...  0. 10. 10.]
 [ 8.  8.  8. ... 10.  0. 10.]
 [ 8.  8.  8. ... 10. 10.  0.]]
Weight matrix is done!!
test data :  noise =  0.1 font : 16
Updating...
test data :  noise =  0.1 font : 16
Updating...
test data :  noise =  0.1 font : 16
Updating...
test data :  noise =  0.1 font : 16
Updating...
test data :  noise =  0.1 font : 16
Updating...
test data :  noise =  0.1 font : 16
Updating...
test data :  noise =  0.1 font : 16
Updating...
test data :  noise =  0.1 font : 16
Updating...
test data :  noise =  0.1 font : 16
Updating...
test data :  noise =  0.1 font : 16
Updating...
test data :  noise =  0.3 font : 16
Updating...
test data : 